# Проект по предобработке данных

- Автор: Лапушкина Ирина
- Дата: 30.08.2025

### Цели и задачи проекта

Цель проекта - подготовить данные о продаже игр для дальнейшего анализа и написания статьи-исследования. 
Статьей планируется привлечь новую аудиторию для игры "Секреты Темнолесья".

Подготовка данных будет включать в себя :
- знакомство с данными, 
- предобработку, 
- выбор необходимого для анализа периода: с 2000 по 2013 год, 
- категоризацию игр на основе оценок пользователей и экспертов,
- выбор топ-7 платформ по количеству выпущенных игр.

### Описание данных

В проекте используем данные из датасета `new_games.csv`, которые содержат информацию о продажах игр разных жанров и платформ, а также пользовательские и экспертные оценки игр:
- `Name` — название игры.
- `Platform` — название платформы.
- `Year of Release` — год выпуска игры.
- `Genre` — жанр игры.
- `NA sales` — продажи в Северной Америке (в миллионах проданных копий).
- `EU sales` — продажи в Европе (в миллионах проданных копий).
- `JP sales` — продажи в Японии (в миллионах проданных копий).
- `Other sales` — продажи в других странах (в миллионах проданных копий).
- `Critic Score` — оценка критиков (от 0 до 100).
- `User Score` — оценка пользователей (от 0 до 10).
- `Rating` — рейтинг организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.

### Содержимое проекта
1. Загрузка данных и знакомство с ними
2. Проверка ошибок в данных и их предобработка
3. Фильтрация данных
4. Категоризация данных
5. Итоговый вывод

---

## 1. Загрузка данных и знакомство с ними

In [1]:
# Импортируем библиотеку pandas
import pandas as pd

In [3]:
# Выводим информацию о датафрейме
games.info()
games_initial_row_count = games.shape[0]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16954 non-null  object 
 1   Platform         16956 non-null  object 
 2   Year of Release  16681 non-null  float64
 3   Genre            16954 non-null  object 
 4   NA sales         16956 non-null  float64
 5   EU sales         16956 non-null  object 
 6   JP sales         16956 non-null  object 
 7   Other sales      16956 non-null  float64
 8   Critic Score     8242 non-null   float64
 9   User Score       10152 non-null  object 
 10  Rating           10085 non-null  object 
dtypes: float64(4), object(7)
memory usage: 1.4+ MB


In [4]:
# Выводим первые 5 строк датафрейма
games.head()

,Name,Platform,Year of Release,Genre,NA sales,EU sales,JP sales,Other sales,Critic Score,User Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


Датафрейм содержит 11 столбцов и 16956 строк. Некоторые столбцы содержат пропуски. 

Названия столбцов написаны в разном стиле, для удобства анализа их стоит привести к стилю snake case.
Кроме того, название столбца Name можно заменить на Game, иначе не понятно о каких именах идет речь.

Изучим типы данных и их корректность:

**Числовые значения с плавающей запятой (float64).** 4 столбца имеют тип данных float64, что подходит для представленных в них данных. Эти столбцы `NA sales`, `Other sales` и `Criric Score`. Для столбца Year of Release данный тип является неверным, так как в столбце представлен год. Соответственно, тип тоже должен быть int64.

**Строковые данные (object).** 7 столбцов имеют тип данных object. Столбцы `Name`, `Platform`, `Genre`, `Rating` содержат строковую информацию (название игры, название платформы, жанр игры и буквенный рейтинг), что логично для текстовых данных. Здесь тип данных object подходит. `EU sales`, `JP sales` содержат информацию о продажах в миллионах копий и рейтинге, представленном дробными числами. Тип данных должен быть изменен на float64. `User Score` нужно привести к int64.

---

## 2.  Проверка ошибок в данных и их предобработка


### 2.1. Названия, или метки, столбцов датафрейма

In [5]:
#Выводим названия всех столбцов
games.columns

Index(['Name', 'Platform', 'Year of Release', 'Genre', 'NA sales', 'EU sales',
       'JP sales', 'Other sales', 'Critic Score', 'User Score', 'Rating'],
      dtype='object')

In [6]:
#Приводим названия столбцов в нижний регистр
games.columns = games.columns.str.lower()
display(games.columns)

Index(['name', 'platform', 'year of release', 'genre', 'na sales', 'eu sales',
       'jp sales', 'other sales', 'critic score', 'user score', 'rating'],
      dtype='object')

In [7]:
#Меняем название name на game. Заменяем пробел на нижнее подчеркивание в названиях столбцов
games = games.rename(columns = {'name': 'game'})
games.columns = games.columns.str.replace(' ', '_')
display(games.columns)

Index(['game', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales',
       'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating'],
      dtype='object')

### 2.2. Типы данных

Неправильный тип данных для некоторых столбцов мог быть присвоен из-за наличия строковых данных в них.

In [8]:
#Исправляем типы данных
for column in ['eu_sales', 'jp_sales', 'user_score']:
    games[column] = pd.to_numeric(games[column], errors = 'coerce')
games['user_score'] = pd.to_numeric(games['user_score'], errors = 'coerce')

games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   game             16954 non-null  object 
 1   platform         16956 non-null  object 
 2   year_of_release  16681 non-null  float64
 3   genre            16954 non-null  object 
 4   na_sales         16956 non-null  float64
 5   eu_sales         16950 non-null  float64
 6   jp_sales         16952 non-null  float64
 7   other_sales      16956 non-null  float64
 8   critic_score     8242 non-null   float64
 9   user_score       7688 non-null   float64
 10  rating           10085 non-null  object 
dtypes: float64(7), object(4)
memory usage: 1.4+ MB


На первом этапе преобразования типов данных мы изменили тип данных с object на float64 в столбцах `eu_sales`, `jp_sales`, `user_score`. Ошибки преобразовали в пропуски.

### 2.2. Наличие пропусков в данных


In [9]:
# Считаем кол-во пропусков
games.isna().sum()

game                  2
platform              0
year_of_release     275
genre                 2
na_sales              0
eu_sales              6
jp_sales              4
other_sales           0
critic_score       8714
user_score         9268
rating             6871
dtype: int64

In [10]:
#Рассчитываем долю пропусков
games.isna().mean()

game               0.000118
platform           0.000000
year_of_release    0.016218
genre              0.000118
na_sales           0.000000
eu_sales           0.000354
jp_sales           0.000236
other_sales        0.000000
critic_score       0.513918
user_score         0.546591
rating             0.405225
dtype: float64

In [11]:
#Смотрим на строки с пропущенным жанром
games[games['genre'].isna()]

,game,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
661,NaN,GEN,1993.0,NaN,1.78,0.53,0.00,0.08,NaN,NaN,NaN
14439,NaN,GEN,1993.0,NaN,0.00,0.00,0.03,0.00,NaN,NaN,NaN


Пропуски встречаются в 8 из 11 стобцов.
Они могли возникнуть из-за того, что данные собирались из открытых источников, а там не всегда присутствует полная информация.

В столбцах `game`и `genre` встречается всего 2 пропуска, что составляет менее 1% от всех данных. К тому же пропуски в обоих столбцах встречаются в одних и тех же строках. Угадать, какие там были значения мы не можем, поэтому стоит удалить эти строки. 

В столбце `year_of_release` пропуски составляют 1,6%. Но здесь мы так же не можем самостоятельно подставить значения, поэтому удаляем эти строки.

В стобцах с продажами `eu_sales` и `jp_sales` пропуски так же составляют менее 1%. Их можно заполнить средними значениями продаж фильмов на той же платформе и вышедших в том же году. 

В столбцах с рейтингом `critic_score`, `user_score` и `rating` очень много пропусков - 40-55%. Удалить эти данные мы не можем. Стоит заменить пропуски на значение-индикатор, которое не будет встречаться в данных, например "-1" для числового рейтинга и "no_rating" для строкового.

In [12]:
#Удаляем пропуски в столбцах с названием игры, жанров и годом выпуска
games = games.dropna(subset=['game', 'genre', 'year_of_release'])

In [13]:
#Заполняем пропуски в столбцах с продажами средним значением
def mean_group_eu_sales(row):
    if pd.isna(row['eu_sales']):
        group = games[(games['platform'] == row['platform']) & 
                   (games['genre'] == row['genre'])]
        return group['eu_sales'].mean()
    else:
        return row['eu_sales']
    
def mean_group_jp_sales(row):
    if pd.isna(row['jp_sales']):
        group = games[(games['platform'] == row['platform']) & 
                   (games['genre'] == row['genre'])]
        return group['jp_sales'].mean()
    else:
        return row['eu_sales']

games['eu_sales'] = games.apply(mean_group_eu_sales, axis=1)
games['jp_sales'] = games.apply(mean_group_jp_sales, axis=1)

In [14]:
#Заполняем пропуски в стоблцах с рейтингом значением-индикатором
games['critic_score'] = games['critic_score'].fillna(-1)
games['user_score'] = games['user_score'].fillna(-1)
games['rating'] = games['rating'].fillna('no_rating')

In [15]:
#Проверяем, что пропусков больше нет
games.isna().sum()

game               0
platform           0
year_of_release    0
genre              0
na_sales           0
eu_sales           0
jp_sales           0
other_sales        0
critic_score       0
user_score         0
rating             0
dtype: int64

In [16]:
#Еще раз преобразовываем тип данных 
games['year_of_release'] = games['year_of_release'].astype('int64')
games['critic_score'] = games['critic_score'].astype('int64')
games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16679 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   game             16679 non-null  object 
 1   platform         16679 non-null  object 
 2   year_of_release  16679 non-null  int64  
 3   genre            16679 non-null  object 
 4   na_sales         16679 non-null  float64
 5   eu_sales         16679 non-null  float64
 6   jp_sales         16679 non-null  float64
 7   other_sales      16679 non-null  float64
 8   critic_score     16679 non-null  int64  
 9   user_score       16679 non-null  float64
 10  rating           16679 non-null  object 
dtypes: float64(5), int64(2), object(4)
memory usage: 1.5+ MB


Заменили пропуски во всех столбцах и привели столбцы `year_of_release`и `critic_score` к типу int64. 

### 2.4. Явные и неявные дубликаты в данных

In [17]:
#Изучаем уникальные значения в категориальных данных
unique_games = games['game'].unique()
display(unique_games)

unique_g = games['genre'].unique()
display(unique_g)

unique_year = games['year_of_release'].unique()
display(unique_year)
 
unique_rat = games['rating'].unique()
display(unique_rat)

array(['Wii Sports', 'Super Mario Bros.', 'Mario Kart Wii', ...,
       'Woody Woodpecker in Crazy Castle 5', 'LMA Manager 2007',
       'Haitaka no Psychedelica'], dtype=object)

array(['Sports', 'Platform', 'Racing', 'Role-Playing', 'Puzzle', 'Misc',
       'Shooter', 'Simulation', 'Action', 'Fighting', 'Adventure',
       'Strategy', 'MISC', 'ROLE-PLAYING', 'RACING', 'ACTION', 'SHOOTER',
       'FIGHTING', 'SPORTS', 'PLATFORM', 'ADVENTURE', 'SIMULATION',
       'PUZZLE', 'STRATEGY'], dtype=object)

array([2006, 1985, 2008, 2009, 1996, 1989, 1984, 2005, 1999, 2007, 2010,
       2013, 2004, 1990, 1988, 2002, 2001, 2011, 1998, 2015, 2012, 2014,
       1992, 1997, 1993, 1994, 1982, 2016, 2003, 1986, 2000, 1995, 1991,
       1981, 1987, 1980, 1983])

array(['E', 'no_rating', 'M', 'T', 'E10+', 'K-A', 'AO', 'EC', 'RP'],
      dtype=object)

В столбцах с годами выпуска неявных дубликатов нет. 
А вот в столбце с рейтингом есть неявный дубликат: "К-А". Это устаревшее название группы Everybody. Значит, "К-А" нужно заменить на "Е".

Кроме того, в столбце с жанрами все значения заадублированы из-за использованияразных регистров. Например, "Misc" и "MISC".

In [18]:
#Нормализуем данные и устраняем неявные дубликаты
games['game'] = games['game'].str.lower()
games['genre'] = games['genre'].str.lower()
games['rating'] = games['rating'].str.upper().replace('K-A', 'E')

In [19]:
#Считаем кол-во явных дубликатов
games.duplicated().sum()

235

В данных содержится 235 явных дубликатов. Их необходимо удалить.

In [20]:
#Удаляем дубликаты
initial_row_count = games.shape[0]

duplicates = games[games.duplicated(keep=False)]
display("Дублирующиеся строки:")
display(duplicates)

games_cleaned = games.drop_duplicates()

final_row_count = games_cleaned.shape[0]

display(f'Количество строк до удаления дубликатов: {initial_row_count}')
display(f'Количество строк после удаления дубликатов: {final_row_count}')

'Дублирующиеся строки:'

,game,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
267,batman: arkham asylum,PS3,2009,action,2.24,1.31,1.31,0.61,91,8.9,T
268,batman: arkham asylum,PS3,2009,action,2.24,1.31,1.31,0.61,91,8.9,T
367,james bond 007: agent under fire,PS2,2001,shooter,1.90,1.13,1.13,0.41,72,7.9,T
368,james bond 007: agent under fire,PS2,2001,shooter,1.90,1.13,1.13,0.41,72,7.9,T
716,god of war: ascension,PS3,2013,action,1.23,0.63,0.63,0.35,80,7.5,M
...,...,...,...,...,...,...,...,...,...,...,...
16799,transformers: prime,Wii,2012,action,0.00,0.01,0.01,0.00,-1,-1.0,NO_RATING
16911,metal gear solid v: the definitive experience,XOne,2016,action,0.01,0.00,0.00,0.00,-1,-1.0,M
16912,metal gear solid v: the definitive experience,XOne,2016,action,0.01,0.00,0.00,0.00,-1,-1.0,M
16939,the longest 5 minutes,PSV,2016,action,0.00,0.00,0.00,0.00,-1,-1.0,NO_RATING


'Количество строк до удаления дубликатов: 16679'

'Количество строк после удаления дубликатов: 16444'

In [21]:
#Считаем кол-во и долю удаленных строк
deleted_rows = games_initial_row_count - games_cleaned.shape[0]
deleted_rows_share = deleted_rows/games_cleaned.shape[0]
display(deleted_rows, deleted_rows_share)

512

0.031135976648017514

В ходе преобразования данных мы удалили 512 строк: 277 строк с пропущенными значениями и 235 дубликатов. Удаленные строки составляют 3% от всех данных.

В ходе преобработки данных были произведены следующие действия:
- названия столбцов приведены к snake case
- название столбца `name` изменено на `game`
- найдено и заменено 277 пропусков в данных
- проведено преобразование типов данных некоторых столбцов. year_of_release, critic_score: тип данных изменен с float64 на int64. eu_sales, jp_sales, user_score: тип данных изменен с object на float64.
- найдено и удалено 235 дублей

---

## 3. Фильтрация данных

In [22]:
#Отбираем период с 2000 по 2013 год
df_actual = games_cleaned[(games_cleaned['year_of_release']>=2000) & (games_cleaned['year_of_release']<=2013)].copy()
df_actual.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12781 entries, 0 to 16954
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   game             12781 non-null  object 
 1   platform         12781 non-null  object 
 2   year_of_release  12781 non-null  int64  
 3   genre            12781 non-null  object 
 4   na_sales         12781 non-null  float64
 5   eu_sales         12781 non-null  float64
 6   jp_sales         12781 non-null  float64
 7   other_sales      12781 non-null  float64
 8   critic_score     12781 non-null  int64  
 9   user_score       12781 non-null  float64
 10  rating           12781 non-null  object 
dtypes: float64(5), int64(2), object(4)
memory usage: 1.2+ MB


In [23]:
#Проверяем корректность фильтрации
min_year = df_actual['year_of_release'].min()
max_year = df_actual['year_of_release'].max()
display(f'Данные за период с {min_year} по {max_year}')

'Данные за период с 2000 по 2013'

Отфильтровали данные за 2000-2013 год. Таких строк оказалось 12781.

---

## 4. Категоризация данных

In [24]:
#Разделяем все игры по оценкам пользователей:
def categorize(score):
    if 0 <= score < 3:
        return 'низкая оценка'
    elif 3 <= score < 8:
        return 'средняя оценка'
    elif 8 <= score <=10:
        return 'высокая оценка'
    else:
        return 'нет оценки'
    
df_actual['user_score_group'] = df_actual['user_score'].apply(categorize)

In [25]:
#Считаем кол-во игр в каждой категории
df_actual.groupby('user_score_group')['game'].count()

user_score_group
высокая оценка    2286
нет оценки        6298
низкая оценка      116
средняя оценка    4081
Name: game, dtype: int64

Больше всего фильмов без пользовательской оценки - 6298. Фильмов с высокой оценкой 6298, со средней - 4081 и с низкой - 116.

In [26]:
#Разделяем все игры по оценкам критиков:
def categorize_2(score):
    if 0 <= score < 30:
        return 'низкая оценка'
    elif 30 <= score < 80:
        return 'средняя оценка'
    elif 80 <= score <=100:
        return 'высокая оценка'
    else:
        return 'нет оценки'
    
df_actual['critic_score_group'] = df_actual['critic_score'].apply(categorize_2)

In [27]:
df_actual.groupby('critic_score_group')['game'].count()

critic_score_group
высокая оценка    1692
нет оценки        5612
низкая оценка       55
средняя оценка    5422
Name: game, dtype: int64

Больше всего фильмов без оценки критиков - 5612. Фильмов с высокой оценкой 1692, со средней - 5422 и с низкой - 55.

In [28]:
#Выделям топ-7 платформ по количеству игр
df_actual['platform'].value_counts().head(7)

PS2     2127
DS      2120
Wii     1275
PSP     1180
X360    1121
PS3     1087
GBA      811
Name: platform, dtype: int64

Топ-7 платформ по количеству игр - это PS2, DS, Wii, PSP, X360, PS3 и GBA.

---

## 5. Итоговый вывод

В конце напишите основной вывод и отразите, какую работу проделали. Не забудьте указать описание среза данных и новых полей, которые добавили в исходный датасет.

Были загружены данные new_games.csv. Они содержат 11 столбцов и 16956 строк, в которых представлена информация о продаже игр, сделанных в разных жанрах и выпущенных на разных платформах, а также пользовательские и экспертные оценки игр. 

При первичном знакомстве с данными и их предобработкой получили такие результаты:

- В восьми столбцах (hotel Name, hotel Address, price for the night, amenities used) были обнаружены пропущенные значения. Максимальное значение пропущенных данных в столбце `user_score` — 55%
- Для оптимизации работы с данными в датафрейме были произведены следующие изменения типов данных:
    - `year_of_release`, `critic_score`: тип данных изменен с float64 на int64,
    - `eu_sales`, `jp_sales`, `user_score`: тип данных изменен с object на float64
- Были отобранны данные за период с 2000 по 2013 год
- Для дополнительной работы с данными были созданы дополнительные столбцы:
    - `user_score_group`: столбец с группами фильмов, созданный на основе пользовательских оценок,
    - `critic_score_group`: столбец с группами фильмов, созданный на основе оценок критиков

Итоги по категоризации на основе оценки пользователей: больше всего фильмов без пользовательской оценки - 6298. Фильмов с высокой оценкой 6298, со средней - 4081 и с низкой - 116.  

Итоги по категоризации на основе оценки критиков: больше всего фильмов без оценки критиков - 5612. Фильмов с высокой оценкой 1692, со средней - 5422 и с низкой - 55.

Выявлен топ-7 платформ по количеству игр - это PS2, DS, Wii, PSP, X360, PS3 и GBA.